In [ ]:
#|default_exp prompt_chain


# Chaining

> Prompt Chaining using Costte and Cosma.

From the [Anthropic Effective Agents Guide](https://www.anthropic.com/research/building-effective-agents)
![](https://www.anthropic.com/_next/image?url=https%3A%2F%2Fwww-cdn.anthropic.com%2Fimages%2F4zrzovbb%2Fwebsite%2F7418719e3dab222dccb379b8879e1dc08ad34c78-2401x1000.png&w=3840&q=75)
Prompt chaining decomposes a task into a sequence of steps, where each LLM call processes the output of the previous one. You can add programmatic checks (see "gate” in the diagram below) on any intermediate steps to ensure that the process is still on track.

**When to use this workflow:**This workflow is ideal for situations where the task can be easily and cleanly decomposed into fixed subtasks. The main goal is to trade off latency for higher accuracy, by making each LLM call an easier task.

Examples where prompt chaining is useful:

- Generating Marketing copy, then translating it into a different language.
- Writing an outline of a document, checking that the outline meets certain criteria, then writing the document based on the outline.

In [ ]:
#| default_exp chain


In [ ]:
#| export
from cosma.core import Agent
from dataclasses import dataclass, field
from typing import List, Callable, Dict, Any, Optional
from fastcore.basics import patch
import re
import json
from cosette import models
from toolslm.xml import json_to_xml, docs_xml, mk_doctype


In [ ]:
from IPython.display import display, Markdown

In [ ]:
#| export
@dataclass
class Chain:
    """A sequence of agents that process information with flexible input/output types and validation"""
    name: str
    agents: List[Agent] = field(default_factory=list)
    steps: List[dict] = field(default_factory=list)
    input_type: str = "text"  # text, image, mixed, structured
    output_type: str = "text"
    validators: List[Callable] = field(default_factory=list)
    
    def __post_init__(self):
        """Initialize chain with type checking and validation setup"""
        self.history = []
        
    @property
    def use(self):
        """Track token usage across chain"""
        return sum((a.chat.use for a in self.agents), start=0)
    
    def validate(self, step_output, step_num):
        """Run validators appropriate for the output type"""
        results = []
        for v in self.validators:
            results.append(v(step_output))
        return all(results)

In [ ]:
#| export
@patch
def __call__(self:Chain, x):
    """Execute the chain on input x"""
    result = x
    for a in self.agents:
        curr_input = result
        result = a.run_with_tools(result)
        self.steps.append(dict(agent=a, input=curr_input, output=result))
    return result

In [ ]:
#| export
@patch
def show(self:Chain):
    """Display chain execution with clear, readable formatting"""
    md = [f"# {self.name}\n"]
    
    # Show chain structure
    md.append("## Structure")
    for i,a in enumerate(self.agents): 
        md.append(f"{i+1}. {a.role}")
        if i < len(self.agents)-1: md.append("   ↓")
    
    # Show execution steps
    if self.steps:
        md.append("\n## Execution")
        for i,s in enumerate(self.steps):
            md.append(f"\nStep {i+1}: {s['agent'].role}")
            md.append(f"\nInput:\n```\n{s['input']}\n```")
            md.append(f"\nOutput:\n```\n{s['output']}\n```")
            if i < len(self.steps)-1: md.append("\n   ↓")
    
    return "\n".join(md)


In [ ]:
# Let's use Cosette to grab a model 
model = models[2]
model

'gpt-4o'

## Basic Chain Composition

The Chain class allows simple composition of AI agents using Python's pipe operator (`|`). This provides a clear, readable way to create processing pipelines.

## Example: Product Description Translation

In this example, we create a chain that:
1. Generates a product description in English
2. Translates it to Spanish
3. Maintains formatting and style

```python
# Create specialized agents
writer = Agent("writer", model, system="Create concise, engaging product descriptions")
translator = Agent("translator", model, system="Translate while maintaining tone and style")

# Compose the chain
chain = Chain("product_translator")
chain.agents = [writer, translator]

# Test with a simple prompt
prompt = "Describe a smart coffee maker"
result = chain(prompt)
```

The chain maintains a history of each step, showing inputs and outputs for debugging and analysis.

In [ ]:
# Create our test agents
writer = Agent("writer", model, system="Create concise, engaging product descriptions")
translator = Agent("translator", model, system="Translate to spanish while maintaining tone and style")

# Simple chain
chain = Chain("product_translator")
chain.agents = [writer, translator]

# Test it
prompt = "Describe a smart coffee maker"
result = chain(prompt)
display(Markdown(chain.show()))


# product_translator

## Structure
1. writer
   ↓
2. translator

## Execution

Step 1: writer

Input:
```
Describe a smart coffee maker
```

Output:
```
Brew the perfect cup every time with our Smart Coffee Maker. Featuring Wi-Fi connectivity, this innovative appliance syncs seamlessly with your smartphone, allowing you to customize brews, set schedules, and receive notifications with ease. With its intuitive touchscreen, voice control compatibility, and auto-cleaning functionality, morning routines have never been more convenient. Ideal for coffee aficionados, it offers precision brewing with programmable strength and temperature settings. Enjoy café-quality coffee at home with smart technology designed to simplify your life.
```

   ↓

Step 2: translator

Input:
```
Brew the perfect cup every time with our Smart Coffee Maker. Featuring Wi-Fi connectivity, this innovative appliance syncs seamlessly with your smartphone, allowing you to customize brews, set schedules, and receive notifications with ease. With its intuitive touchscreen, voice control compatibility, and auto-cleaning functionality, morning routines have never been more convenient. Ideal for coffee aficionados, it offers precision brewing with programmable strength and temperature settings. Enjoy café-quality coffee at home with smart technology designed to simplify your life.
```

Output:
```
Prepara la taza perfecta cada vez con nuestra Cafetera Inteligente. Con conectividad Wi-Fi, este innovador aparato se sincroniza sin problemas con tu smartphone, permitiéndote personalizar las preparaciones, establecer horarios y recibir notificaciones con facilidad. Con su pantalla táctil intuitiva, compatibilidad con control por voz y función de autolimpieza, las rutinas matutinas nunca han sido más convenientes. Ideal para los aficionados al café, ofrece una preparación precisa con ajustes programables de intensidad y temperatura. Disfruta de café de calidad de cafetería en casa con tecnología inteligente diseñada para simplificar tu vida.
```

## Structured Data with Chains

Following Anthropic's best practices for handling structured data, we can create chains that process and validate XML-formatted information. This approach offers several benefits:

1. **Clear Data Structure**: Using XML tags provides explicit structure to agent outputs
2. **Validation**: Easy to verify required fields and format
3. **Consistent Parsing**: Reliable transformation between formats

### Basic Example: Product Analysis Chain

We'll create a chain that:
- Analyzes product descriptions
- Outputs structured XML data
- Validates the structure
- Transforms data as needed

In [ ]:
# 1. Create the analyzer agent
analyzer = Agent("product_analyzer", model,
    system="""You are a product analyzer. When analyzing products, always provide detailed, structured analysis using XML tags. Be thorough but concise.""")


In [ ]:
# 2. Create a function to format the prompt
def create_analysis_prompt(product_description: str) -> str:
    return f"""Here is the format to use for your analysis:

<documents>
<document index="1">
<source>product analysis</source>
<document_content>
    <features>List key features</features>
    <pricing>Price range and considerations</pricing>
    <market>Target market segments</market>
</document_content>
</document>
</documents>

Analyze this product: {product_description}"""

In [ ]:
# Create chain with structured output
analysis_chain = Chain("product_analyzer")
analysis_chain.agents = [analyzer]

# Test with a product
test_prompt = "Analyze this product: A smart home security camera with AI detection, 4K resolution, and cloud storage."
result = analysis_chain(test_prompt)
display(Markdown(analysis_chain.show()))


# product_analyzer

## Structure
1. product_analyzer

## Execution

Step 1: product_analyzer

Input:
```
Analyze this product: A smart home security camera with AI detection, 4K resolution, and cloud storage.
```

Output:
```
```xml
<ProductAnalysis>
    <ProductName>Smart Home Security Camera</ProductName>
    <Features>
        <Feature>
            <Name>AI Detection</Name>
            <Description>
                The camera incorporates artificial intelligence to distinguish between different types of motion events.
                This helps in reducing false alarms by only notifying the user of significant activities, such as identifying
                human figures versus animals or passing vehicles.
            </Description>
        </Feature>
        <Feature>
            <Name>4K Resolution</Name>
            <Description>
                Offers ultra-high-definition video quality, providing clear and detailed images that are essential for
                identifying faces and other critical details, enhancing the effectiveness of security monitoring.
            </Description>
        </Feature>
        <Feature>
            <Name>Cloud Storage</Name>
            <Description>
                Provides remote storage solutions where recorded footage is automatically uploaded to the cloud.
                This feature ensures that video evidence is preserved even if the camera is damaged or stolen.
                Various storage plans might be available, offering different capacities and retention periods.
            </Description>
        </Feature>
    </Features>
    <Benefits>
        <Benefit>
            <Description>Enhanced security with AI enabling smarter alert systems and decreased false positives.</Description>
        </Benefit>
        <Benefit>
            <Description>Better image quality with 4K resolution ensures critical details aren't missed.</Description>
        </Benefit>
        <Benefit>
            <Description>Remote access to footage from anywhere, thanks to cloud storage solutions.</Description>
        </Benefit>
    </Benefits>
    <Considerations>
        <Consideration>
            <Description>Requires a stable internet connection for effective AI processing and cloud storage management.</Description>
        </Consideration>
        <Consideration>
            <Description>May incur additional costs related to cloud storage subscriptions beyond initial purchase.</Description>
        </Consideration>
        <Consideration>
            <Description>High-resolution video files (4K) will consume more bandwidth and storage space than lower resolutions.</Description>
        </Consideration>
    </Considerations>
</ProductAnalysis>
```

```

In [ ]:
#| export
def extract_xml(text: str, tag: str) -> str:
    """Extract content from XML tags, including CDATA sections"""
    pattern = f'<{tag}>\s*(?:<!\[CDATA\[)?(.*?)(?:\]\]>)?\s*</{tag}>'
    match = re.search(pattern, text, re.DOTALL)
    return match.group(1).strip() if match else ""

In [ ]:
def create_analysis_prompt(product_description: str) -> str:
    return f"""Here is the format to use for your analysis:

<documents>
<document index="1">
<source>product analysis</source>
<document_content>
    <features>
    - Key feature 1
    - Key feature 2
    </features>
    <pricing>
    - Price range
    - Key pricing factors
    </pricing>
    <market>
    - Target market 1
    - Target market 2
    </market>
</document_content>
</document>
</documents>

Analyze this product: {product_description}"""

In [ ]:
# First run our analysis
test_product = "A smart home security camera with AI detection, 4K resolution, and cloud storage."
result = analysis_chain(create_analysis_prompt(test_product))

# Now extract from the nested structure
document_content = extract_xml(result, 'document_content')
features = extract_xml(document_content, 'features')
pricing = extract_xml(document_content, 'pricing')
market = extract_xml(document_content, 'market')

# Show the extracted content
print("Features:\n", features)
print("\nPricing:\n", pricing)
print("\nMarket:\n", market)

Features:
 - AI Detection for advanced motion recognition to minimize false alerts
                - 4K Resolution providing detailed and crystal-clear video footage
                - Cloud Storage offering secure, remote access to recorded content

Pricing:
 - Price range: $150 to $400
                - Key pricing factors: Quality of AI technology, cost of cloud storage subscription, and inclusion of features such as night vision or two-way audio

Market:
 - Target market 1: Tech-savvy homeowners interested in smart security solutions
                - Target market 2: Small businesses needing robust surveillance systems


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()